# Trees
#### Basic binary tree
Implement a nodal tree structure and methods to identify:
- if a node is a leaf
- the values of a node's children
- the values of its grandchildren
- the size of it's subree (the node and all of its descendants)
- the height of its subtree.

In [2]:
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def is_leaf(self):
        return not self.left and not self.right

    def children(self):
        c = []
        if self.left:
            c.append(self.left.val)
        if self.right:
            c.append(self.right.val)
        return c

    def grandchildren(self):
        g = []
        b = ['left', 'right']
        for child in [self.left, self.right]:
            if child and child.left:
                g.append(child.left.val)
            if child and child.right:
                g.append(child.right.val)
        return g

    def subtree_size(self):
        def component_size(node):
            if node:
                return 1 + component_size(node.left) + component_size(node.right)
            else:
                return 0
        return component_size(self)
    
    def subtree_height(self):
        levels = set()
        def child_check(current_node, l):
            if current_node:
                levels.add(l)
                child_check(current_node.left, l+1)
                child_check(current_node.right, l+1)
            return
        child_check(self,1)
        return max(levels)

In [3]:
my_node = Node(5)
print(f"First node value: {my_node.val}")
print(f"Is first node a leaf? {my_node.is_leaf()}")
print("Adding more nodes to tree...")
my_node.left = Node('l')
my_node.left.right = Node('R')
my_node.right = Node('r')
my_node.right.right = Node('R')
print(f"Is first node still a leaf? {my_node.is_leaf()}")
print(f"What are the first node's children's values? {my_node.children()}")
print(f"And the grandchildren's values? {my_node.grandchildren()}")
print(f"The size of the subtree is now {my_node.subtree_size()} and the height is {my_node.subtree_height()}")
print(f"And the size of the right child's subtree is {my_node.right.subtree_size()} and the height is {my_node.right.subtree_height()}")


First node value: 5
Is first node a leaf? True
Adding more nodes to tree...
Is first node still a leaf? False
What are the first node's children's values? ['l', 'r']
And the grandchildren's values? ['R', 'R']
The size of the subtree is now 5 and the height is 3
And the size of the right child's subtree is 2 and the height is 2


What if we need to find the parent node? The following implementation returns...
- whether a node is the root
- the IDs of all ancestors
- the depth of a node
- the lowest common ancestor (LCA) of two nodes
- the distance, or number of edges in the path, bettween two nodes

In [5]:
class Node:
    def __init__(self, id, parent=None, left=None, right=None):
        self.id = id
        self.parent = parent
        self.left = left
        self.right = right

    def is_root(self):
        return not self.parent

    def ancestors(self):
        a = []
        curr = self
        while curr.parent:
            a.append(curr.parent.id)
            curr = curr.parent
        return a

    def depth(self):
        d = 0
        curr = self
        while curr.parent:
            d += 1
            curr = curr.parent
        return d

    def lca(self, node2):
        d = self.depth()
        d2 = node2.depth()
        curr, curr2 = self, node2
        while d2 > d:
            curr2 = curr2.parent
            d2 -= 1
        while d > d2:
            curr = curr.parent
            d -= 1
        while curr != curr2:
            if not curr.parent:
                raise Exception("No common ancestor")
            curr, curr2 = curr.parent, curr2.parent
        return curr.id

    def distance(self, node2):
        d1, d2 = self.depth(), node2.depth()
        d = d1 + d2
        curr, curr2 = self, node2
        while d2 > d1:
            curr2 = curr2.parent
            d2 -= 1
        while d1 > d2:
            curr = curr.parent
            d1 -= 1
        while curr != curr2:
            if not curr.parent:
                raise Exception("Nodes not connected")
            d1 -= 1
            d2 -= 1
            curr, curr2 = curr.parent, curr2.parent
        return d - d1 - d2       

In [6]:
my_node = Node('a')
print(f"First node id: {my_node.id}")
print(f"Is first node the root? {my_node.is_root()}")
print("Adding more nodes to tree...")
node_b = Node('b',parent=my_node)
my_node.left = node_b
my_node.left.left = Node('d', parent=my_node.left)
node_e = Node('e', parent=node_b)
node_b.right = node_e
my_node.right = Node('c', parent=my_node)
node_f = Node('f', parent=my_node.right)
my_node.right.left = node_f
node_h = Node('h', parent=node_b.left)
node_b.left.left = node_h
my_node.left.left.right = Node('i', parent=node_b.left)
my_node.right.left.left = Node('j', parent=node_f)
node_j = my_node.right.left.left
my_node.right.left.right = Node('k', parent=node_f)
print(f"Is first node still the root? {my_node.is_root()}")
print(f"Is node_j a root? {node_j.is_root()}")
print(f"What are node_j's ancestors? {node_j.ancestors()}")
print(f"And node_h's ancestors? {node_h.ancestors()}")
print(f"What are the depths of nodes a, b, f and h? a:{my_node.depth()}, b:{node_b.depth()}, f:{node_f.depth()}, h:{node_h.depth()}")
print(f"LCA between h and e is {node_h.lca(node_e)}; distance is {node_h.distance(node_e)}")
print(f"LCA between j and f is {node_j.lca(node_f)}; distance is {node_j.distance(node_f)}")

First node id: a
Is first node the root? True
Adding more nodes to tree...
Is first node still the root? True
Is node_j a root? False
What are node_j's ancestors? ['f', 'c', 'a']
And node_h's ancestors? ['d', 'b', 'a']
What are the depths of nodes a, b, f and h? a:0, b:1, f:2, h:3
LCA between h and e is b; distance is 3
LCA between j and f is f; distance is 1


#### Aligned chain
Given a binary tree, return the length of the longest descendant chain of nodes whose ids align with their depth.

In [8]:
# Example binary tree
new = Node(7)
new.left = Node(1, parent=new)
new.left.left = Node(2, parent=new.left)
new.left.left.left = Node(4, parent=new.left.left)
new.left.left.right = Node(3, parent=new.left.left)
new.left.right = Node(8, parent=new.left)
new.right = Node(3, parent=new)
new.right.left = Node(2, parent=new.right)
new.right.left.left = Node(3, parent=new.right.left)
new.right.left.right = Node(3, parent=new.right.left)

In [9]:
# Solution uses a recrsive function that makes a preorder traversal of the tree from the root
def aligned_chain(node, chain_length=0, max_chain=0, depth=0):
    
    if not node:
        #print(f'node not found, returning max_chain ', max_chain)
        return max_chain
    #print(f'running function on node {node.id} with parameters cl={chain_length}, mc={max_chain}, d={depth}')
    if node.id == depth:
        #print(f'node id = depth = {depth}. We have a winner!')
        chain_length +=1
        #print(f'chain incremented to ', chain_length)
        max_chain = max(max_chain, chain_length)
        #print(f'max_chain now ', max_chain)
    else:
        chain_length = 0
        #print(f'node id != depth. Chain length reset to 0')
    depth += 1
    #print(f'depth incremented to ', depth)
    #print(f'will now run function on left and right children of node {node.id}')
    return max(aligned_chain(node.left, chain_length, max_chain, depth), aligned_chain(node.right, chain_length, max_chain, depth))

In [10]:
aligned_chain(new)

3

#### Hidden message
A message is hidden in the nodes of a binary tree. Each node has two letters: b, i or a, followed by part of the message. The first letter determines the ordering:
- If the first character is b, it goes before its left subtree, which goes before the right subtree (preorder).
- If a, the node goes after its right subtree and the right subtree goes after the left subtree (postorder).
- If i, the node goes after its left subtree and before its right (inorder).

In [12]:
# Example binary tree
message = Node('bn')
message.left = Node('i_', parent=message)
message.left.left = Node('ae', parent=message.left)
message.left.left.left = Node('bi', parent=message.left.left)
message.left.left.right = Node('bc', parent=message.left.left)
message.left.right = Node('it', parent=message.left)
message.right = Node('a!', parent=message)
message.right.left = Node('br', parent=message.right)
message.right.left.right = Node('ay', parent=message.right.left)

In [13]:
# A more computationally efficient version of this solution would generate an array, rather than a string, 
# since adding to a string takes O(len(string)) complexity whereas adding to an array is only O(1). 
# Doing this would require embedding a recursive function into a non-recursive function (which would join the letters from the array),
# rather than having a single recursive function.

def hidden_message(root):
    if not root:
        #print('Node does not exist, returning empty string')
        return ''
    if root.id[0] not in ['b','a','i']:
        raise Exception(f'Invalid node id: {root.id}')
    if root.id[0] == 'b':
        #print(f'We have a b-node! Will run function recursively on child nodes then return {root.id[1]} + left + right')
        return root.id[1] + hidden_message(root.left) + hidden_message(root.right)
    if root.id[0] == 'a':
        #print(f'We have an a-node! Will run function recursively on child nodes then return left + right + {root.id[1]}')
        return hidden_message(root.left) + hidden_message(root.right) + root.id[1]
    if root.id[0] == 'i':
        #print(f'We have an i-node! Will run function recursively on child nodes then return left + {root.id[1]} + right')        
        return hidden_message(root.left) + root.id[1] + hidden_message(root.right) 

In [14]:
hidden_message(message)

'nice_try!'

#### Aligned path
Given a binary tree, return the length of the longest path (not necessarily descendant) where node value aligns with depth.

In [16]:
def aligned_path(node, depth=0):
    if not node:
        return 0
    if node.id == depth:
        return 1 + aligned_path(node.left, depth+1) + aligned_path(node.right, depth+1)
    else:
        return max(aligned_path(node.left, depth+1), aligned_path(node.right, depth+1))


In [17]:
aligned_path(new)

3

#### Tree layout
Given the root of a binary tree, layout the tree on a grid as follows:
- the root is at $(r,c) = (0,0)$
- left subtrees are placed one cell below their parents $(r+1)$
- right subtrees are placed one cell right of their parents $(c+1)$

If nodes are laid on the same coordinates they are stacked. Return the maximum number of nodes stacked on the same coordinate.

In [19]:
def layout(node):
    position_count = dict()
    def visit(node, r, c):
        if not node:
            return
        if not (r, c) in position_count:
            position_count[(r, c)] = 1
        else:
            position_count[(r, c)] += 1
        visit(node.left, r+1, c)
        visit(node.right, r, c+1)

    visit(node, 0, 0)
    max_stack = 0
    return max(position_count.values()) 

In [20]:
layout(new)

2

#### Triangle count
Count the number of triangles in a tree, given the root. A triangle is a set of three nodes, a, b, c, where 
- a is the lowest common ancestor of b and c
- b and c have the same depth
- the paths from b and c to a consist respectively of only left and only right children.

In [22]:
def triangle_count(root):
    count = 0
    if not root:
        return count
    curr_r = curr_l = root
    while curr_l.left and curr_r.right:
        count += 1
        curr_l = curr_l.left
        curr_r = curr_r.right
        
    return count + triangle_count(root.left) + triangle_count(root.right)

In [23]:
# Example binary tree
new = Node(7)
new.left = Node(1, parent=new)
new.left.right = Node(2, parent=new.left)
new.left.right.left = Node(3, parent=new.left.left)
new.left.right.right = Node(8, parent=new.left)
new.right = Node(3, parent=new)
new.right.left = Node(2, parent=new.right)
new.right.left.left = Node(3, parent=new.right.left)
new.right.right = Node(4, parent=new.right.left)
new.right.right.right = Node(3, parent=new.right.left)

In [24]:
triangle_count(new)

4

#### Invert a binary tree
Such that what was left is right and what was right is left...

In [26]:
def invert_tree(node):
    if not node:
        return
    node.left, node.right = node.right, node.left
    invert_tree(node.left)
    invert_tree(node.right)

In [27]:
invert_tree(new)

In [28]:
print(f"D0: {new.id}")
print(f"D1: left={new.left.id}, right={new.right.id}")
print(f"D2: left-left={new.left.left.id}, left-right={new.left.right.id}, right-left={new.right.left.id}")
print(f"D3: left-left-left={new.left.left.left.id}, left-right-right={new.left.right.right.id}, right-left-left={new.right.left.left.id}, right-left-right={new.right.left.right.id}")

D0: 7
D1: left=3, right=1
D2: left-left=4, left-right=2, right-left=2
D3: left-left-left=3, left-right-right=3, right-left-left=8, right-left-right=3


#### Evaluate expression tree
Given a tree that has either mathematical functions of integers as nodes, evaluate it as a mathematical expression. The tree is not binary and nodes may have more than two children.

In [30]:
class Math_Node:
    def __init__(self, kind, num, children):
        self.kind = kind # sum, product, max, min, or num
        self.num = num # only valid when kind == num
        self.children = children # only valid when kind != num

In [31]:
# Example tree
math_root = Math_Node('min',None,[])
math_root.children.append(Math_Node('max', None,[]))
math_root.children[0].children.append(Math_Node('num', 4, None))
math_root.children[0].children.append(Math_Node('num', 6, None))
math_root.children[0].children.append(Math_Node('sum', None, []))
math_root.children[0].children[2].children.append(Math_Node('num', 5, None))
math_root.children[0].children[2].children.append(Math_Node('num', 7, None))
math_root.children.append(Math_Node('sum', None,[]))
math_root.children[1].children.append(Math_Node('product', None, []))
math_root.children[1].children[0].children.append(Math_Node('num', 6, None))
math_root.children[1].children[0].children.append(Math_Node('num', 8, None))

In [32]:
def evaluate_exp(root):
    if root.kind == 'sum':
        return sum([evaluate_exp(x) for x in root.children])
    elif root.kind == 'product':
        res = 1
        for child in root.children:
            res *= evaluate_exp(child)
        return res
    elif root.kind == 'max':
        return max([evaluate_exp(x) for x in root.children])
    elif root.kind == 'min':
        return min([evaluate_exp(x) for x in root.children])
    elif root.kind == 'num':
        return root.num
    else:
        raise ValueError("root kind not valid for mathematical expression tree")



In [33]:
evaluate_exp(math_root)

12

### Breadth-first search
BFS traversals move level by level, visiting all nodes in a level before moving onto the next. To do this, they use a queue structure, rather than recursive iteration, adding children to one end of the queue while visiting each node in order from the other end.

In [35]:
# Queue
class QueueNode:
    def __init__(self, val):
        self.val = val
        self.next = None

class Queue:
    def __init__(self):
        self.head = None
        self.tail = None

    def empty(self):
        return not self.head

    def push(self, val):
        new = QueueNode(val)
        if self.tail:
            self.tail.next = new
        else:
            self.head = new
        self.tail = new

    def pop(self):
        if self.head:
            val = self.head.val
            self.head = self.head.next
            if not self.head:
                self.tail = None
            return val    

In [36]:
# Node depth queue recipe for level-order traversal
def node_depth_queue_recipe(root):
    Q = Queue()
    Q.push((root, 0)) # second element of tuple represents depth
    while not Q.empty():
        node, depth = Q.pop()
        if not node:
            continue
        # Do something with node and depth
        print(f"Current node: {node.id}, depth={depth}")
        Q.push((node.left, depth+1))
        Q.push((node.right, depth+1))
    

In [37]:
node_depth_queue_recipe(new)

Current node: 7, depth=0
Current node: 3, depth=1
Current node: 1, depth=1
Current node: 4, depth=2
Current node: 2, depth=2
Current node: 2, depth=2
Current node: 3, depth=3
Current node: 3, depth=3
Current node: 8, depth=3
Current node: 3, depth=3


#### Left view
Given the root of a binary tree, return a list of the left-most node of each level.

In [39]:
def left_view(root):
    leftmost_nodes = [root.id]
    next_level = 1
    Q = Queue()
    Q.push((root, 0))
    while not Q.empty():
        node, level = Q.pop()
        if node:
            if level == next_level:
                leftmost_nodes.append(node.id)
                next_level += 1
            Q.push((node.left, level + 1))
            Q.push((node.right, level + 1))
    return leftmost_nodes

In [40]:
left_view(new)

[7, 3, 4, 3]